In [ ]:
# Importar bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor

In [ ]:
# Carregar os dados do CSV
file_path = 'dados.csv'
df = pd.read_csv(file_path, sep=';', parse_dates=['Data'], dayfirst=True, index_col='Data', encoding='ISO-8859-1')

In [ ]:
# Separe as variáveis independentes
X = df.loc[:,['Receitas_Correntes_PMF','Preço_Médio_Venda_Imóvel_Residencial_FIPEZAP','ICV_UDESC',
              'Massa_Salarial_SC_IBGE_6474','Rendimento_Nominal_IBGE_6472','População_Fpolis_IBGE_5917',
              'Consumo_MWh_Celesc','Exportações_MDIC','Importações_MDIC'
                   ]]

In [ ]:
# Normalizar os dados de X e trasformar em Dataframe
scaler = StandardScaler()
X_normalizado = scaler.fit_transform(X)
X_normalizado = pd.DataFrame(X_normalizado, columns=X.columns, index=X.index)

In [ ]:
# Separe variável dependente (PIB)
y = df.loc[:,['PIB_Fpolis_IBGE_5938']]

In [ ]:
# Normalizar os dados de y e transformar em Dataframe
scaler = StandardScaler()
y_normalizado = scaler.fit_transform(y)
y_normalizado = pd.DataFrame(y_normalizado, columns=y.columns, index=y.index)

In [ ]:
# Separar em treino e predição
X_normalizado_treino = X_normalizado.loc['2012-03-01':'2021-12-01']
X_normalizado_predicao = X_normalizado.loc['2022-03-01':]

y_normalizado_treino = y_normalizado.loc['2012-03-01':'2021-12-01']
y_normalizado_treino_1d = y_normalizado_treino.values.ravel()

In [ ]:
# Treine modelo MLP e preveja
model = MLPRegressor(
    activation='relu',
    hidden_layer_sizes=(50,),
    max_iter=500,
    solver='adam',
    random_state=15,
    alpha=0.001
)
model.fit(X_normalizado_treino, y_normalizado_treino_1d)
y_predicao = model.predict(X_normalizado_predicao)

In [ ]:
# Desnormalizando todas as previsões
mean_pib = df['PIB_Fpolis_IBGE_5938'].mean()
std_pib = df['PIB_Fpolis_IBGE_5938'].std()
y_desnormalizado = mean_pib + (y_predicao * std_pib)

In [ ]:
# Plotar o gráfico
plt.figure(figsize=(14, 5))
plt.plot(y_normalizado_treino.index, y_normalizado_treino, label='PIB_Fpolis', linestyle='--', marker='x')
plt.plot(X_normalizado_predicao.index, y_predicao, label='PIB_Fpolis', linestyle='--', marker='x')
plt.title('Comparação entre Valores Reais e Previstos do PIB')
plt.xlabel('Data')
plt.ylabel('PIB (desnormalizado)')
plt.legend()
plt.grid(True)
plt.show()

mse = mean_squared_error(y_normalizado_treino, model.predict(X_normalizado_treino))
rmse = np.sqrt(mse)
r2 = r2_score(y_normalizado_treino, model.predict(X_normalizado_treino))

print(f"RMSE: {rmse}")
print(f"R²: {r2}")

In [ ]:
# Exibir dados da previsão
datas_previsao = X_normalizado_predicao.index
df_previsao = pd.DataFrame({
    'Trimestre': datas_previsao,
    'PIB_previsao': y_desnormalizado
})
pd.options.display.float_format = '{:,.0f}'.format
print(df_previsao)

In [ ]:
# Opcional: Buscar melhores hiperparâmetros pro treino do modelo
param_grid = {
    'hidden_layer_sizes': [(100,), (50, 50), (100, 50)],
    'activation': ['relu', 'tanh', 'logistic'],
    'solver': ['adam', 'sgd'],
    'max_iter': [200, 500, 1000],
    'alpha': [0.0001, 0.001, 0.01, 0.1, 1],
    'random_state': [15] }


grid_search = GridSearchCV(MLPRegressor(), param_grid, cv=5, n_jobs=-1, scoring='neg_mean_squared_error')
grid_search.fit(X_normalizado_treino, y_normalizado_treino_1d)

print(f"Best parameters: {grid_search.best_params_}")
print(f"Best cross-validation score: {grid_search.best_score_}")

In [ ]:
# Opcional: Treinar modelo para cada valor de random_state de 1 a 42
for random_state in range(1, 43):
    # Treine o modelo de MLPRegressor com o random_state atual
    model = MLPRegressor(
        activation='relu',
        hidden_layer_sizes=(50,),
        max_iter=500,
        solver='adam',
        random_state=random_state,
        alpha=0.001
    )
    model.fit(X_normalizado_treino, y_normalizado_treino_1d)

    # Previsão e comparação
    y_predicao = model.predict(X_normalizado_predicao)

    # Desnormalizando as previsões
    mean_pib = df['PIB_Fpolis_IBGE_5938'].mean()
    std_pib = df['PIB_Fpolis_IBGE_5938'].std()
    pib_desnormalizado = mean_pib + (y_predicao * std_pib)

    # Desnormalizar os valores reais de PIB
    y_normalizado_treino_desnormalizado = mean_pib + (y_normalizado_treino.values * std_pib)

    # Plotar o gráfico
    plt.figure(figsize=(14, 2))
    plt.plot(y_normalizado_treino.index, y_normalizado_treino_desnormalizado, label='PIB_Fpolis (real)', linestyle='--', marker='x')
    plt.plot(X_normalizado_predicao.index, pib_desnormalizado, label=f'PIB_Fpolis (previsto) - random_state={random_state}', linestyle='-', marker='o')
    plt.title(f'Comparação entre Valores Reais e Previstos do PIB (random_state={random_state})')
    plt.xlabel('Data')
    plt.ylabel('PIB (desnormalizado)')
    plt.legend()
    plt.grid(True)
    plt.show()

    # Cálculo das métricas
    mse = mean_squared_error(y_normalizado_treino, model.predict(X_normalizado_treino))
    rmse = np.sqrt(mse)
    r2 = r2_score(y_normalizado_treino, model.predict(X_normalizado_treino))

    print(f"random_state={random_state}")
    print(f"RMSE: {rmse}")
    print(f"R²: {r2}")
    print("-" * 200)